In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    # Prompt the user to enter the API key if not set
    import getpass
    api_key = getpass.getpass("Please enter your OPENAI_API_KEY: ")
    if not api_key:
        raise ValueError("OPENAI_API_KEY variable is not set in the environment variables or provided by user.")

In [4]:
from agents import Agent, Runner, TResponseInputItem

simple_agent = Agent(
    name="Malik Shb ka Agent",
    instructions="You are a helpful assistant. Reply very consisely.",
    model="gpt-4o-mini"
)

conversation: list[TResponseInputItem] = []

print("You are chating with Malik Shb ka Agent. Type 'exit' to quit.\n")
while True:
    user_input = input("You: ")
    print("You: "+ user_input)

    conversation.append({"role": "user", "content": user_input})
    if user_input.lower() == "exit":
        print("Exiting the chat. Goodbye!")
        break

    response = await Runner.run(simple_agent, conversation)
    print("Agent: " + response.final_output + "\n")
    conversation.append({"role": "assistant", "content": response.final_output})






You are chating with Malik Shb ka Agent. Type 'exit' to quit.

You: hey
Agent: Hello! How can I assist you today?

You: tell me about the president of Italy?
Agent: As of my last update, the President of Italy is Sergio Mattarella, in office since February 3, 2015. He serves as the head of state and represents national unity. His role is largely ceremonial, with powers including appointing the Prime Minister and overseeing the functioning of government.

You: capital of italy
Agent: The capital of Italy is Rome.

You: what was my second msg?
Agent: Your second message was asking about the president of Italy.

You: great man
Agent: Glad you think so! Would you like to know more about him?

You: tell me about yourself
Agent: I’m an AI assistant designed to help with information and tasks. How can I assist you today?

You: thank you please exit
Agent: You're welcome! If you need anything else, feel free to return. Have a great day!

You: exit
Exiting the chat. Goodbye!


In [5]:
print(conversation)

[{'role': 'user', 'content': 'hey'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}, {'role': 'user', 'content': 'tell me about the president of Italy?'}, {'role': 'assistant', 'content': 'As of my last update, the President of Italy is Sergio Mattarella, in office since February 3, 2015. He serves as the head of state and represents national unity. His role is largely ceremonial, with powers including appointing the Prime Minister and overseeing the functioning of government.'}, {'role': 'user', 'content': 'capital of italy'}, {'role': 'assistant', 'content': 'The capital of Italy is Rome.'}, {'role': 'user', 'content': 'what was my second msg?'}, {'role': 'assistant', 'content': 'Your second message was asking about the president of Italy.'}, {'role': 'user', 'content': 'great man'}, {'role': 'assistant', 'content': 'Glad you think so! Would you like to know more about him?'}, {'role': 'user', 'content': 'tell me about yourself'}, {'role': 'assistant', 'conten

In [10]:
from agents import Agent, Runner, handoff, RunContextWrapper

history_tutor_agent = Agent(
    name="History Tutor Agent",
    handoff_description="Specialist in assisting with history-related topics.",
    instructions="""
    You are a history tutor. Your job is to help students understand historical events, figures, and concepts.
    """,
)

math_tutor_agent = Agent(
    name="Math Tutor Agent",
    handoff_description="Specialist in assisting with math-related topics.",
    instructions="""
    You are a math tutor. Your job is to help in mathematical concepts, problem-solving, and calculations.
    Explain concepts step-by-step and provide examples where necessary.
    """,
)

def on_math_handoff(context: RunContextWrapper):
    print("Handoff to Math Tutor Agent")
    return context

def on_history_handoff(context: RunContextWrapper):
    print("Handoff to History Tutor Agent")
    return context

# Triage agent that decides which tutor to handoff to based on the topic
triage_agent = Agent(
    name="Triage Agent",
    instructions="""
    You are a triage agent. Your job is to determine whether a student's question is related to history or math.
    If the question is related to history, handoff to the History Tutor Agent.
    If the question is related to math, handoff to the Math Tutor Agent.
    If the question is unrelated to either subject, provide a general response."
    """,
    handoffs=[handoff(history_tutor_agent, on_handoff=on_history_handoff), 
              handoff(math_tutor_agent, on_handoff=on_math_handoff)],
)

conversation: list[TResponseInputItem] = []
last_agent = triage_agent

print("You are chating with Triage Agent. Type 'exit' to quit.\n")
while True:
    user_input = input("You: ")
    print("You: "+ user_input)

    if user_input.lower() == "exit":
        print("Exiting the chat. Goodbye!")
        break

    conversation.append({"role": "user", "content": user_input})
    response = await Runner.run(triage_agent, conversation)

    conversation = response.to_input_list()
    last_agent = response.last_agent

    conversation.append({"role": "assistant", "content": response.final_output})

    print(f"{last_agent.name}: " + response.final_output + "\n")



You are chating with Triage Agent. Type 'exit' to quit.

You: hi
Triage Agent: Hello! How can I help you today? Are you looking for help with history, math, or something else?

You: 12 multiply by 5
Handoff to Math Tutor Agent
Math Tutor Agent: Let's solve \( 12 \times 5 \):

Step-by-step:
1. Multiply 12 by 5:
   \( 12 \times 5 = 60 \)

So, \( 12 \times 5 = 60 \).

If you have more multiplication questions or want to know how to do it with larger numbers, let me know!

You: founder of pakistan
Handoff to History Tutor Agent
History Tutor Agent: The founder of Pakistan is **Muhammad Ali Jinnah**.

He was a prominent lawyer, politician, and leader of the All-India Muslim League. Jinnah played a crucial role in the creation of Pakistan in 1947, serving as its first Governor-General until his death in 1948. He is often referred to as **"Quaid-e-Azam"**, which means "Great Leader."

If you'd like to know more about Muhammad Ali Jinnah or the events leading up to the creation of Pakistan, ju